In [36]:
from solid import *
from solid.utils import *

class Pallet:
    def __init__(self, width, depth):
        self.width = width
        self.depth = depth
        self.tallness = 0
        self.layers = []

    def add_layer(self, item_width, item_depth, item_height, x_count, y_count):

        if x_count < 2:
            x_offset = 0
        else:
            x_offset = item_width + (self.width-(item_width*x_count))/(x_count-1)

        if y_count < 2:
            y_offset = 0
        else:
            y_offset = item_depth + (self.depth-(item_depth*y_count))/(y_count-1)

        layer = cube([item_width, item_depth, item_height])
        layer = translate([0,0,self.tallness])(layer)
        for x in range(x_count):
            for y in range(y_count):
                item = cube([item_width, item_depth, item_height])
                item = translate([x * x_offset, y * y_offset, self.tallness])(item)
                layer += item

        self.layers.append(layer)
        self.tallness += item_height

    def render(self, file_name="pallet.scad"):
        out = union()(*self.layers)
        scad_render_to_file(out, file_name)


In [37]:
# Create a pallet with a width of 120 and a depth of 120
pallet = Pallet(120, 120)

# Add three layers to the pallet
pallet.add_layer(item_width=20, item_depth=120, item_height=2, x_count=3, y_count=1)
pallet.add_layer(item_width=20, item_depth=20, item_height=10, x_count=3, y_count=3)
pallet.add_layer(item_width=120, item_depth=20, item_height=2, x_count=1, y_count=3)
pallet.add_layer(item_width=20, item_depth=120, item_height=2, x_count=5, y_count=1)

# Render the pallet to a file
pallet.render(file_name="example_pallet.scad")

In [38]:
# proceed